In [27]:
import numpy as np
import pandas as pd

ncd_df = pd.read_csv("results/vega-lite/vega-lite/ncd_costs.csv", names=["study", "notation", "from_slug", "to_slug", "a", "b", "ab"])
ncd_df["cd"] = ncd_df["ab"] - ncd_df[["a", "b"]].min(axis=1)
ncd_df["ncd"] = (1000 * ncd_df["cd"] / ncd_df[["a", "b"]].max(axis=1))
square = ncd_df.pivot_table(index="from_slug", columns="to_slug", values="cd").fillna(0)

difflib_df = pd.read_csv("results/difflib_costs.csv", names=["study", "notation", "from_slug", "to_slug", "difflib"])
square = difflib_df.pivot_table(index="from_slug", columns="to_slug", values="difflib").fillna(0)
order = list(square.index)

def check(X):
    ok = 0
    results = []
    for i in range(len(X)):
        for j in range(len(X)):
            for k in range(len(X)):
                if X[i,j] > X[i,k]+X[k,j]:
                    results.append((i,j,k,order[i],order[j],order[k],X[i,j],X[i,k],X[k,j]))
                else:
                    ok += 1
    return results, ok

#results, ok = check(square.values)
#print(100.0*len(results)/ok)

In [28]:
from numba import njit


@njit
def find_edges(dmat):
    n = len(dmat)
    result = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            has_k = False
            direct = dmat[i, j]
            if direct != 0:
                for k in range(n):
                    if k == i or k == j:
                        continue
                    via_k = dmat[i, k] + dmat[k, j]
                    if (via_k - direct) / direct <= 0:
                        has_k = True
                        break
                if not has_k:
                    result[i, j] = direct
    return result

In [23]:
edges = find_edges(square.values)

In [25]:
np.count_nonzero(edges)

0

In [26]:
edges

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])